# Imports

In [2]:
import pandas as pd
import numpy as np

In [3]:
def reduce_df(df:pd.DataFrame):
    # df[df['area']==0.05]
    return df.groupby('metric').aggregate('mean')[['comp', 'suff']].reset_index()

int_metric_map = {
    'electricity': ['mae', 'mse'],
    'traffic': ['mae', 'mse'],
    'mimic_iii': ['auc', 'cross_entropy']
}

test_metric_map = {
    'electricity': ['mae', 'mse'],
    'traffic': ['mae', 'mse'],
    'mimic_iii': ['auc', 'accuracy']
}

datasets = ['electricity', 'traffic', 'mimic_iii']
models = ['DLinear', 'MICN', 'SegRNN', 'Crossformer']
attr_methods = [
    'feature_ablation', 'augmented_occlusion', 
    'feature_permutation', 'winIT', 'tsr' ,'wtsr'
]

short_form = {
    'feature_ablation': 'FA',
    'occlusion':'FO',
    'augmented_occlusion': 'AFO',
    'feature_permutation': 'FP',
    'winIT': 'WinIT',
    'tsr':'TSR',
    'wtsr': 'WinTSR'
}
NUM_ITERATIONS = 3

# Summarize

In [16]:
results = []
for dataset in datasets:
    for attr_method in attr_methods:
        for metric in int_metric_map[dataset]:
            for model in models:
                for itr_no in range(1, NUM_ITERATIONS+1):
                    df = pd.read_csv(f'results/{dataset}_{model}/{itr_no}/{attr_method}.csv')
                    df = reduce_df(df)
                    comp, suff= df[df['metric']==metric][['comp', 'suff']].values[0]
                    
                    results.append([
                        dataset, attr_method, metric, model, itr_no, comp, suff
                    ])

result_df = pd.DataFrame(results, columns=['dataset', 'attr_method', 'metric', 'model', 'itr_no', 'comp', 'suff'])
result_df.head(3)

,dataset,attr_method,metric,model,itr_no,comp,suff
0,electricity,feature_ablation,mae,DLinear,1,10.860365,12.558040
1,electricity,feature_ablation,mae,DLinear,2,10.433981,12.210812
2,electricity,feature_ablation,mae,DLinear,3,10.852396,12.406175


In [17]:
result_df = result_df.groupby(['dataset', 'attr_method', 'metric', 'model'])[['comp', 'suff']].mean().reset_index()
result_df = result_df[result_df['metric'].isin(['mae', 'auc'])]

selected = result_df['metric'].isin(['auc', 'accuracy'])
result_df.loc[selected, ['comp', 'suff']] = 1 - result_df[selected][['comp', 'suff']]

In [47]:
result_df['comp_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['comp'].rank(ascending=False)
result_df['suff_rank'] = result_df.groupby(['dataset', 'metric', 'model'])['suff'].rank(ascending=True)
# result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].mean().reset_index()

In [ ]:
result_df.groupby(['dataset', 'metric', 'attr_method'])[['comp_rank', 'suff_rank']].std().round(1).reset_index()

In [48]:
df = pd.concat([
    result_df.drop(columns='suff_rank').rename(columns={'comp_rank': 'rank'}), 
    result_df.drop(columns='comp_rank').rename(columns={'suff_rank': 'rank'})
], axis=0)

In [61]:
df.groupby(['dataset', 'metric', 'attr_method'])['rank'].mean().round(1).reset_index()

,dataset,metric,attr_method,rank
0,electricity,mae,augmented_occlusion,4.1
1,electricity,mae,feature_ablation,3.0
2,electricity,mae,feature_permutation,6.0
3,electricity,mae,tsr,1.9
4,electricity,mae,winIT,4.9
5,electricity,mae,wtsr,1.1
6,mimic_iii,auc,augmented_occlusion,4.2
7,mimic_iii,auc,feature_ablation,2.2
8,mimic_iii,auc,feature_permutation,5.6
9,mimic_iii,auc,tsr,2.8


In [63]:
df.groupby(['dataset', 'metric', 'attr_method'])['rank'].std().round(2).reset_index()

,dataset,metric,attr_method,rank
0,electricity,mae,augmented_occlusion,0.35
1,electricity,mae,feature_ablation,0.00
2,electricity,mae,feature_permutation,0.00
3,electricity,mae,tsr,0.35
4,electricity,mae,winIT,0.35
5,electricity,mae,wtsr,0.35
6,mimic_iii,auc,augmented_occlusion,1.67
7,mimic_iii,auc,feature_ablation,1.39
8,mimic_iii,auc,feature_permutation,0.52
9,mimic_iii,auc,tsr,1.16


,dataset,metric,attr_method,comp_rank,suff_rank
0,electricity,mae,augmented_occlusion,0.5,0.0
1,electricity,mae,feature_ablation,0.0,0.0
2,electricity,mae,feature_permutation,0.0,0.0
3,electricity,mae,tsr,0.0,0.5
4,electricity,mae,winIT,0.5,0.0
5,electricity,mae,wtsr,0.0,0.5
6,mimic_iii,auc,augmented_occlusion,1.2,1.9
7,mimic_iii,auc,feature_ablation,1.2,1.7
8,mimic_iii,auc,feature_permutation,0.5,0.0
9,mimic_iii,auc,tsr,0.8,1.0
